<a href="https://colab.research.google.com/github/MaRwaLaHmami/classification_detection_tumeurs/blob/main/tumeur_projet_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import os
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"marwalahmami1","key":"0d46f230084885994b12b3ef85b46f42"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 🔹 Télécharger le dataset
!pip install kaggle
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

# 🔹 Décompresser le fichier
import zipfile

dataset_path = "/content/brain-tumor-mri-dataset.zip"
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")

print("✅ Dataset extrait avec succès !")

# 🔹 Vérifier la structure des fichiers
import os

dataset_dir = "/content/dataset"
for root, dirs, files in os.walk(dataset_dir):
    print(f"📂 Dossier : {root} | 📌 {len(files)} fichiers")
    print("-" * 50)

Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
 99% 147M/149M [00:06<00:00, 27.6MB/s]
100% 149M/149M [00:06<00:00, 23.8MB/s]
✅ Dataset extrait avec succès !
📂 Dossier : /content/dataset | 📌 0 fichiers
--------------------------------------------------
📂 Dossier : /content/dataset/Testing | 📌 0 fichiers
--------------------------------------------------
📂 Dossier : /content/dataset/Testing/meningioma | 📌 306 fichiers
--------------------------------------------------
📂 Dossier : /content/dataset/Testing/pituitary | 📌 300 fichiers
--------------------------------------------------
📂 Dossier : /content/dataset/Testing/glioma | 📌 300 fichiers
--------------------------------------------------
📂 Dossier : /content/dataset/Testing/notumor | 📌 405 fichiers
--------------------------------------------------
📂 Dossier : /content/dataset/Training | 📌 0 fichiers
--------------------------------------------------
📂 Dossier : /content/datas

In [ ]:
#Définition correcte des chemins
images_path = "/content/dataset/Training"
masks_path = "/content/dataset/Testing"


In [ ]:
import numpy as np
import cv2
import os
from glob import glob

def load_data(images_path, masks_path, img_size=256):
    images = []
    masks = []

    # 📌 Chercher les fichiers dans tous les sous-dossiers
    images_files = sorted(glob(os.path.join(images_path, '*', '*.jpg')))
    masks_files = sorted(glob(os.path.join(masks_path, '*', '*.jpg')))

    print(f"📌 Nombre total d'images trouvées : {len(images_files)}")
    print(f"📌 Nombre total de masques trouvés : {len(masks_files)}")

    if len(images_files) == 0 or len(masks_files) == 0:
        print("⚠️ ERREUR : Aucun fichier trouvé. Vérifie les chemins des images et des masques !")
        return np.array([]), np.array([]), {}, {}

    def normalize_name(filename):
        return filename.replace("Tr-", "").replace("Te-", "")

    image_names = {normalize_name(os.path.splitext(os.path.basename(f))[0]): f for f in images_files}
    mask_names = {normalize_name(os.path.splitext(os.path.basename(f))[0]): f for f in masks_files}

    common_files = set(image_names.keys()).intersection(set(mask_names.keys()))
    print(f"✅ Nombre de paires valides trouvées : {len(common_files)}")

    for file_name in common_files:
        img_file = image_names[file_name]
        mask_file = mask_names[file_name]

        # 🔹 Charger et redimensionner l'image **EN RGB (3 canaux)**
        img = cv2.imread(img_file)  # Lire en couleur (BGR par défaut)
        img = cv2.resize(img, (img_size, img_size))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir en RGB
        img = img / 255.0  # Normalisation
        images.append(img)

        # 🔹 Charger et redimensionner le masque (garder en grayscale)
        mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (img_size, img_size))
        mask = mask / 255.0
        masks.append(mask)

    # 📌 Convertir en tableaux numpy
    images = np.array(images).reshape(-1, img_size, img_size, 3)  # 🔥 3 canaux maintenant ✅
    masks = np.array(masks).reshape(-1, img_size, img_size, 1)

    return images, masks, image_names, mask_names

# 🔹 Charger les données avec correction
X, y, image_names, mask_names = load_data(images_path, masks_path)

# 🔹 Vérifier les nouvelles dimensions des images et masques
print(f"📌 Taille des images chargées (DOIT ÊTRE 3 CANAUX) : {X.shape}")
print(f"📌 Taille des masques chargés : {y.shape}")
# import numpy as np
# import cv2
# import os
# from glob import glob

# def load_data(images_path, masks_path, img_size=256):
#     images = []
#     masks = []

#     # Chercher les fichiers dans tous les sous-dossiers
#     images_files = sorted(glob(os.path.join(images_path, '*', '*.jpg')))
#     masks_files = sorted(glob(os.path.join(masks_path, '*', '*.jpg')))

#     print(f"📌 Nombre total d'images trouvées : {len(images_files)}")
#     print(f"📌 Nombre total de masques trouvés : {len(masks_files)}")

#     if len(images_files) == 0 or len(masks_files) == 0:
#         print("⚠️ ERREUR : Aucun fichier trouvé. Vérifie les chemins des images et des masques !")
#         return np.array([]), np.array([]), {}, {}

#     # Normaliser les noms des fichiers en supprimant "Tr-" et "Te-"
#     def normalize_name(filename):
#         return filename.replace("Tr-", "").replace("Te-", "")

#     # Associer les fichiers d’images et de masques (sans préfixes)
#     image_names = {normalize_name(os.path.splitext(os.path.basename(f))[0]): f for f in images_files}
#     mask_names = {normalize_name(os.path.splitext(os.path.basename(f))[0]): f for f in masks_files}

#     # Trouver les fichiers correspondants
#     common_files = set(image_names.keys()).intersection(set(mask_names.keys()))
#     print(f"✅ Nombre de paires valides trouvées : {len(common_files)}")

#     for file_name in common_files:
#         img_file = image_names[file_name]
#         mask_file = mask_names[file_name]

#         # Charger et redimensionner l'image
#         img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
#         img = cv2.resize(img, (img_size, img_size)) / 255.0
#         images.append(img)

#         # Charger et redimensionner le masque
#         mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
#         mask = cv2.resize(mask, (img_size, img_size)) / 255.0
#         masks.append(mask)

#     # Convertir en tableaux numpy
#     images = np.array(images).reshape(-1, img_size, img_size, 1)
#     masks = np.array(masks).reshape(-1, img_size, img_size, 1)

#     return images, masks, image_names, mask_names

# # Charger les données avec correction
# X, y, image_names, mask_names = load_data(images_path, masks_path)

# # Vérifier les tailles des images et masques
# print(f"📌 Taille des images chargées : {X.shape}")
# print(f"📌 Taille des masques chargés : {y.shape}")

# # Vérifier les noms des fichiers après correction
# print("📌 Exemples de noms d'images :", list(image_names.keys())[:10])
# print("📌 Exemples de noms de masques :", list(mask_names.keys())[:10])
# print(f"✅ Nombre de paires valides trouvées : {len(set(image_names.keys()).intersection(set(mask_names.keys())))}")


📌 Nombre total d'images trouvées : 5712
📌 Nombre total de masques trouvés : 1311
✅ Nombre de paires valides trouvées : 1311
📌 Taille des images chargées (DOIT ÊTRE 3 CANAUX) : (1311, 256, 256, 3)
📌 Taille des masques chargés : (1311, 256, 256, 1)


In [ ]:
from sklearn.model_selection import train_test_split

# 🔹 Diviser les données en 80% entraînement et 20% validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔹 Vérifier les tailles des ensembles
print(f"📌 Taille de X_train : {X_train.shape}")
print(f"📌 Taille de X_val : {X_val.shape}")
print(f"📌 Taille de y_train : {y_train.shape}")
print(f"📌 Taille de y_val : {y_val.shape}")


📌 Taille de X_train : (1048, 256, 256, 3)
📌 Taille de X_val : (263, 256, 256, 3)
📌 Taille de y_train : (1048, 256, 256, 1)
📌 Taille de y_val : (263, 256, 256, 1)


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, concatenate, Input

def build_vgg_unet():
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    # Geler les poids de VGG16
    for layer in vgg16.layers:
        layer.trainable = False

    # 📌 Extraire les features du modèle pré-entraîné
    s1 = vgg16.get_layer('block1_conv2').output
    s2 = vgg16.get_layer('block2_conv2').output
    s3 = vgg16.get_layer('block3_conv3').output
    s4 = vgg16.get_layer('block4_conv3').output
    b1 = vgg16.get_layer('block5_conv3').output

    # 📌 Décodeur (Upsampling + Concaténation)
    def decoder_block(input_tensor, num_filters):
        x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
        x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
        x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
        return x

    d1 = decoder_block(b1, 512)
    d1 = concatenate([d1, s4])

    d2 = decoder_block(d1, 256)
    d2 = concatenate([d2, s3])

    d3 = decoder_block(d2, 128)
    d3 = concatenate([d3, s2])

    d4 = decoder_block(d3, 64)
    d4 = concatenate([d4, s1])

    # 📌 Couche de sortie
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(d4)

    return Model(vgg16.input, outputs, name="VGG_UNet")

# 🔹 Construire et compiler le modèle
model = build_vgg_unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 🔹 Afficher l'architecture du modèle
model.summary()


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "VGG_UNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1 (Conv2D)     │ (None, 256, 256, 64)   │          1,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2 (Conv2D)     │ (None, 256, 256, 64)   │         36,928 │ block1_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_pool               │ (None, 128, 128, 64)   │              0 │ block1_conv2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_conv1 (Conv2D)     │ (None, 128, 128, 128)  │         73,856 │ block1_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_conv2 (Conv2D)     │ (None, 128, 128, 128)  │        147,584 │ block2_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_pool               │ (None, 64, 64, 128)    │              0 │ block2_conv2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_conv1 (Conv2D)     │ (None, 64, 64, 256)    │        295,168 │ block2_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_conv2 (Conv2D)     │ (None, 64, 64, 256)    │        590,080 │ block3_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_conv3 (Conv2D)     │ (None, 64, 64, 256)    │        590,080 │ block3_conv2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_pool               │ (None, 32, 32, 256)    │              0 │ block3_conv3[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_conv1 (Conv2D)     │ (None, 32, 32, 512)    │      1,180,160 │ block3_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_conv2 (Conv2D)     │ (None, 32, 32, 512)    │      2,359,808 │ block4_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_conv3 (Conv2D)     │ (None, 32, 32, 512)    │      2,359,808 │ block4_conv2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_pool               │ (None, 16, 16, 512)    │              0 │ block4_conv3[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block5_conv1 (Conv2D)     │ (None, 16, 16, 512)    │      2,359,808 │ block4_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block5_conv2 (Conv2D)     │ (None, 16, 16, 512)    │      2,359,808 │ block5_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block5_conv3 (Conv2D)

 Total params: 23,409,409 (89.30 MB)

 Trainable params: 8,694,721 (33.17 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Assurez-vous que ces imports sont bien faits
# 📌 Vérifier que la fonction load_data() est bien définie
try:
    load_data
except NameError:
    print("⚠️ La fonction load_data() n'est pas définie. Assurez-vous d'exécuter sa cellule.")

# 📌 Vérifier si X et y existent, sinon les recharger
if 'X' not in locals() or 'y' not in locals():
    print("⚠️ X et y non définis, rechargement des données...")
    X, y, _, _ = load_data(images_path, masks_path)
    print("✅ Données chargées avec succès !")

# 📌 Vérifier et rediviser les données si nécessaire
if 'X_train' not in locals() or 'y_train' not in locals():
    from sklearn.model_selection import train_test_split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    print("✅ Données redivisées !")

# 📌 Vérification des dimensions avant l'entraînement
print(f"📌 Taille de X_train : {X_train.shape}, Taille de y_train : {y_train.shape}")
print(f"📌 Taille de X_val : {X_val.shape}, Taille de y_val : {y_val.shape}")

# 📌 Entraîner le modèle
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=16)


📌 Taille de X_train : (1048, 256, 256, 3), Taille de y_train : (1048, 256, 256, 1)
📌 Taille de X_val : (263, 256, 256, 3), Taille de y_val : (263, 256, 256, 1)
Epoch 1/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 4573s 69s/step - accuracy: 0.1345 - loss: 0.7587 - val_accuracy: 0.1504 - val_loss: 0.4749
Epoch 2/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 4576s 69s/step - accuracy: 0.1571 - loss: 0.4569 - val_accuracy: 0.1504 - val_loss: 0.4491
Epoch 3/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 4542s 69s/step - accuracy: 0.1574 - loss: 0.4510 - val_accuracy: 0.1504 - val_loss: 0.4437
Epoch 4/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 4569s 69s/step - accuracy: 0.1477 - loss: 0.4441 - val_accuracy: 0.1504 - val_loss: 0.4499
Epoch 5/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 4576s 69s/step - accuracy: 0.1573 - loss: 0.4442 - val_accuracy: 0.1504 - val_loss: 0.4401
Epoch 6/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 4506s 68s/step - accuracy: 0.1499 - loss: 0.4434 - val_accuracy: 0.1504 - val_loss: 0.4418
Epoch 7/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 4548s 69s/step - accuracy: 0